In [3]:
from ipyleaflet import Map, basemaps, Polygon, Marker, Popup
from ipywidgets import HTML
import IPython.core.display as ip
import shapely.geometry as sg

In [4]:
import pickle
from scipy.spatial import ConvexHull
import networkx as nx
import numpy as np

In [5]:
center = (23.0225, 72.5714)
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=10)

In [6]:
with open('region_sim1.p', 'rb') as fp:
    S5, S10, S20 = pickle.load(fp)

poly = Polygon(
    locations=[S20['patch'][j] for j in ConvexHull(S20['patch']).vertices],
    color="green",
    fill_color="green",
    weigth=3
)

popup = Popup(
    location=S20['patch'][0],
    child=HTML(value="Region 1"),
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)
m.add_layer(popup)

m.add_layer(poly)

with open('region_sim2.p', 'rb') as fp:
    S5, S10, S20 = pickle.load(fp)

poly = Polygon(
    locations=[S20['patch'][j] for j in ConvexHull(S20['patch']).vertices],
    color="green",
    fill_color="green",
    weigth=3
)

popup = Popup(
    location=S20['patch'][0],
    child=HTML(value="Region 2"),
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)
m.add_layer(popup)
m.add_layer(poly)

with open('region_sim3.p', 'rb') as fp:
    S5, S10, S20 = pickle.load(fp)

poly = Polygon(
    locations=[S20['patch'][j] for j in ConvexHull(S20['patch']).vertices],
    color="green",
    fill_color="green",
    weigth=3
)

popup = Popup(
    location=S20['patch'][0],
    child=HTML(value="Region 3"),
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)
m.add_layer(popup)
m.add_layer(poly)

with open('region_sim4.p', 'rb') as fp:
    S5, S10, S20 = pickle.load(fp)

poly = Polygon(
    locations=[S20['patch'][j] for j in ConvexHull(S20['patch']).vertices],
    color="green",
    fill_color="green",
    weigth=3
)
popup = Popup(
    location=S20['patch'][0],
    child=HTML(value="Region 4"),
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)
m.add_layer(popup)
m.add_layer(poly)

In [7]:
# Map showing the region's 20000 population patches
# Will rrequire a rerun
m

Map(center=[23.0225, 72.5714], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [8]:
center = (23.0225, 72.5714)
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=10)

In [7]:
G = nx.read_graphml("../../Full_network.graphml", node_type=int, edge_key_type=int)

In [11]:
with open("../dashboard/data/wards_updated.p", "rb") as fp:
    ward_poly = pickle.load(fp)

In [12]:
ward_area = {}
for w in ward_poly:
    ward_area[w] = sg.Polygon(ward_poly[w]['polygon'][0]).area

In [13]:
ward_nodes = {}
for w in ward_poly:
    arr = []
    poly = sg.MultiPoint(ward_poly[w]['polygon'][0]).convex_hull
    for j in G:
        point = sg.Point(G.nodes[j]['X'],G.nodes[j]['Y'])
        if poly.contains(point):
            arr.append(j)
    ward_nodes[w] = arr

NameError: name 'G' is not defined

In [10]:
Pop = nx.get_node_attributes(G, 'NP')

NameError: name 'G' is not defined

In [12]:
ward_density = {}
for w in ward_area:
    total_pop = 0
    for node in ward_nodes[w]:
        total_pop+=Pop[node]
    ward_density[w] = float(total_pop)/ward_area[w]

In [13]:
keys = list(ward_area.keys())

In [10]:
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = ((value-minimum)**2 / (maximum - minimum)**2)*240
    return "hsl("+str(ratio)+", 100%, 50%)"

In [15]:
rank = np.argsort([-ward_density[k] for k in keys])+1

In [16]:
for i in range(64):
    poly = Polygon(
        locations=[j[::-1] for j in ward_poly[keys[i]]['polygon'][0]],
        color="green",
        fill_color=rgb(1,64, rank[i]),
        weigth=0,
        opacity=0,
        fill_opacity=0.5
    )
    m.add_layer(poly)

In [17]:
m

Map(center=[23.0225, 72.5714], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [18]:
center = (23.0225, 72.5714)
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=10)

In [7]:
with open('../dashboard/data/ward_ts.p', 'rb') as fp:
    ward_ts = pickle.load(fp)

In [8]:
rank = np.argsort([-ward_ts[k][-1][1] for k in keys])+1

NameError: name 'keys' is not defined

In [21]:
for i in range(64):
    poly = Polygon(
        locations=[j[::-1] for j in ward_poly[keys[i]]['polygon'][0]],
        color="green",
        fill_color=rgb(1,64, rank[i]),
        weigth=0,
        opacity=0,
        fill_opacity=0.5
    )
    m.add_layer(poly)

In [22]:
m

Map(center=[23.0225, 72.5714], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [15]:
arr = np.array(sorted([ward_ts[k][-1][1] for k in ward_ts], reverse=True)+[0]).reshape((5,13))

In [16]:
for i in range(5):
    print(arr[i][0], arr[i][-1], rgb(1, 64, i*13+1), rgb(1, 64, i*13+13))

806 207 hsl(0.0, 100%, 50%) hsl(8.70748299319728, 100%, 50%)
206 147 hsl(10.219198790627361, 100%, 50%) hsl(37.79289493575208, 100%, 50%)
145 104 hsl(40.876795162509445, 100%, 50%) hsl(87.3167044595616, 100%, 50%)
97 72 hsl(91.97278911564626, 100%, 50%) hsl(157.27891156462584, 100%, 50%)
71 0 hsl(163.50718065003778, 100%, 50%) hsl(247.67951625094486, 100%, 50%)


In [23]:
center = (23.0225, 72.5714)
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=10)

In [24]:
Cmat = np.load("../../Cmat.npy")
with open("../../mapping.bin", "rb") as fp:
    mapping = pickle.load(fp)

In [25]:
ward_final = {}
for w in ward_area:
    total_pop = 0
    coeff = 0
    for node in ward_nodes[w]:
        total_pop+=Pop[node]
        for jnode in ward_nodes[w]:
            coeff+=np.exp(-Cmat[mapping[node]][mapping[jnode]])*Pop[node]*Pop[jnode]
    coeff = float(coeff)/(total_pop**2)
    ward_final[w] = (ward_area[w], coeff, total_pop)

In [26]:
rank = np.argsort([-ward_final[w][2]*np.log(ward_final[w][2])/(ward_final[w][1]**2*ward_final[w][0]) for w in keys])+1

In [27]:
for i in range(64):
    poly = Polygon(
        locations=[j[::-1] for j in ward_poly[keys[i]]['polygon'][0]],
        color="green",
        fill_color=rgb(1,64, rank[i]),
        weigth=0,
        opacity=0,
        fill_opacity=0.5
    )
    m.add_layer(poly)

In [28]:
m

Map(center=[23.0225, 72.5714], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…